In [1]:
    #STD
    std_list = []
    for i in range(3):
        std_list.append(np.std(dataset[i]))

    
    #RMS Error
    rms_error_list = []
    for i in range(3):
        sum_list = []
        for j in range(len(dataset[i])):
            sum_list.append((dataset[i][j]-rms_list[i])**2)
        rms_error_list.append(math.sqrt(sum(sum_list)/len(sum_list)))


        rms_total_error = 0
        for i in range(3):
            rms_total_error += ((rms_list[i] * (rms_list[0]**2 + rms_list[1]**2 + rms_list[2]**2)**-0.5)*rms_error_list[i])**2
        rms_total_error = math.sqrt(rms_total_error)

        rmsr_error_list = []
        for i in range(3):
            numerator = 0
            for x in range(3):
                if x != i:
                    numerator += rms_list[x] **2
            denominator = (rms_list[0]**2 + rms_list[1]**2 + rms_list[2]**2)**1.5
            rmsr_error_list.append(numerator/denominator)

        return rms_list, total_rms, rmsr_list, sway_area, npl, swayrange, std_list, rms_error_list, rms_total_error, rmsr_error_list


NameError: name 'np' is not defined

In [2]:
#FUNCTION TO IMPORT ACCELEROMETER DATA
import csv
import os
import numpy as np
import scipy.stats as stats
import random
import math
import matplotlib.pyplot as plt


print(os.getcwd())
my_directory = '/Users/syt/Documents/GitHub/hsl_python/SpaceVestibExperiment' 
folder = 'ak1' #change this based on person

def import_accel(rootdir, foldername):

    directory = rootdir + '/' + foldername + '/Acc' 

    os.chdir(directory)

    for r, d, files in os.walk(directory):
        for filename in files:
            if 'si' in filename and 'up' in filename:
                si_up = np.loadtxt(filename, delimiter = ',', skiprows=1, usecols = (1,2,3,4,5,6))
            elif 'si' in filename and 'p' in filename:
                si_p = np.loadtxt(filename, delimiter = ',', skiprows=1, usecols = (1,2,3,4,5,6))
            elif 'st' in filename and 'up' in filename: 
                st_up = np.loadtxt(filename, delimiter = ',', skiprows=1, usecols = (1,2,3,4,5,6))
            elif 'st' in filename and 'p' in filename: 
                st_p = np.loadtxt(filename, delimiter = ',', skiprows=1, usecols = (1,2,3,4,5,6))    
    return si_up.transpose(), si_p.transpose(), st_up.transpose(), st_p.transpose()

ak_accel = import_accel(my_directory, folder) #ak_accel[0] = 6x400 matrix (only up1,sit)


/Users/apara/Desktop


FileNotFoundError: [Errno 2] No such file or directory: '/Users/syt/Documents/GitHub/hsl_python/SpaceVestibExperiment/ak1/Acc'

In [3]:
#These are all Vedant's stats functions from the shimmer file
def scaled_correlation_time(signal1, signal2):
    signal1 = (signal1 - np.mean(signal1))/np.std(signal1)
    signal2 = (signal2 - np.mean(signal2))/np.std(signal2)
    acorr = np.correlate(signal1, signal2, mode='full')
    acorr = acorr[(acorr.size // 2 ):] / np.max(acorr)
#     plt.plot(acorr)
    tau = np.argmax([acorr < 1/np.exp(1)])
    return tau / len(acorr)

def signal_statistics(signal):
    mean = np.mean(signal)
    std = np.std(signal)
    skewness = stats.skew(signal)
    kurtosis = stats.kurtosis(signal)
    maximum = np.max(signal)
    minimum = np.min(signal)
    iqr = stats.iqr(signal)
    variation = stats.variation(signal)
    entropy = stats.entropy(np.abs(signal))
    corrtime = scaled_correlation_time(signal,signal)        
    return np.asarray([mean, std, skewness, kurtosis, maximum, minimum, variation, entropy])

def spectrum_statistics(signal):
    fs,pxx = sig.periodogram(signal, fs = 50, nfft = 1000, scaling = 'density', detrend = 'constant')
#   plt.plot(fs,pxx)
    #plt.xlim(0,0.1)
    peak = fs[np.argmax(pxx)]
    peakmag = np.max(pxx)
    integral = np.trapz(pxx,fs)
    energy = np.dot(pxx,pxx)
    shannon = np.sum(pxx*np.log(1/pxx))
    return [peak, peakmag, integral, energy, shannon]


In [4]:
#outputs a 4*6*8 array -- all the signal stats (all 4 phases, all 6 directions) per person

def create_array(person):
    signal_list = []
    for run in range(4):
        for measure in range(6):
            new_list_measure = [signal_statistics(ak_accel[run][measure])]
            signal_list.append(new_list_measure)

    signal_arr = np.asarray(signal_list)
    signal_arr = signal_arr.reshape(4,6,8)

    return signal_arr


In [5]:
i = 3 #input number from 0-3; representing UP-SI, P-SI, UP-ST, P-ST
j = 3 #input number from 0-5; representing x,y,z linear and x,y,z rotational

plt.plot(ak_accel[i][j])
plt.show()


NameError: name 'ak_accel' is not defined